Since we are using nessie as part of this here test we need to start a local version of that. When it is up you should be able to got to this here local address and see something running. 
Address: (nessie)[http://localhost:19120/]. Be aware that we will restart the container - we are not stopping it. 

In [3]:
!!docker rm $(docker ps -aq --filter name=nessie-test)
!!docker run -d --name nessie-test -p 19120:19120 -p 9000:9000 quay.io/projectnessie/nessie:0.104.2

['d5719fb14396af7dade739124cdefe940a0ae90b6c232f6fc3fc12cf0c7adfcd']

First we set the SPARK_HOME Env to make sure we can use SPARK locally

In [4]:
import os
spark_home = "/home/kasper/developer/utils/spark"
os.environ['SPARK_HOME'] = spark_home
data_path = os.path.abspath(os.getcwd())

In [5]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
    .appName("Nessie_Iceberg_Setup")
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.9.1,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.103.3")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions")
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://localhost:19120/api/v1")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.authentication.type", "NONE")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.warehouse", f"file://{data_path}/test_data/")
    .getOrCreate())

25/06/20 11:27:29 WARN Utils: Your hostname, kasper-ThinkPad-P52 resolves to a loopback address: 127.0.1.1; using 10.242.149.19 instead (on interface wlp0s20f3)
25/06/20 11:27:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/kasper/developer/utils/spark-3.5.6-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/kasper/.ivy2/cache
The jars for the packages stored in: /home/kasper/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8d13c563-bc75-4013-b11e-178a1b7967f7;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.1 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.103.3 in central
:: resolution report :: resolve 114ms :: artifacts dl 7ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.1 from central in [default]
	org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.103.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf     

First we create some sample data

In [ ]:
df = spark.createDataFrame([(1, "a"), (2, "b")], ["id", "value"])
df.show()

Save the dataframe as a parquet file.

In [ ]:
df.write.parquet("test_data/users")

Now we read the files created - and add them to nessie

In [6]:
df_user = spark.read.parquet("test_data/users")
df_user.show()

+---+-----+
| id|value|
+---+-----+
|  2|    b|
|  1|    a|
+---+-----+



In [14]:
df_user.write.format("iceberg").mode("append").saveAsTable("nessie.users")

Now we have a table in nessie - now lets create a query on that.

In [15]:
spark.sql("SELECT * FROM nessie.users").show()

+---+-----+
| id|value|
+---+-----+
|  2|    b|
|  1|    a|
+---+-----+

